In [1]:
import geography_based_batched
import geography_based
from datasets import *
import torch.utils.data as data
import torchvision.transforms as transforms


In [2]:
transform_train = transforms.Compose([           
        transforms.ToTensor(),                          
        ])

In [3]:
dataset = YfccPlacesDataset(transform_train, 10)


subsetting first 1500!


In [4]:
dataloader = data.DataLoader(dataset=dataset, 
              num_workers=0,
              batch_size=1,
              collate_fn=collate_fn)

In [6]:
len(dataset)


1500

In [7]:
import time


In [10]:
times = []
batch_sz = []
for i in [4,8,16,32,64,128,256,512]:
    start = time.time()
    batched_res = count_langs_batch(dataloader, i)
    end = time.time()
    print(end - start)
    times.append(end - start)
    batch_sz.append(i)
    
    

batch method



  0%|          | 0/1500 [00:00<?, ?it/s]

4


/n/fs/revise-scr/alex/miniconda2/envs/toolenv/lib/python3.7/site-packages/torch/nn/functional.py:2506: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
100%|██████████| 1500/1500 [06:13<00:00,  4.02it/s] 


377.6066975593567
batch method


...

400.9880955219269
batch method



  0%|          | 0/1500 [00:00<?, ?it/s]

16


 55%|█████▌    | 831/1500 [04:17<03:27,  3.23it/s] 


KeyboardInterrupt: 

## Compare results 

In [ ]:
old_res = count_langs_old(dataloader)

In [ ]:
batched_res.keys()

In [ ]:
batched_res['country_with_imgs'].keys()

In [ ]:
test = (old_res['country_with_imgs']['Japan'][0])[0][0]
test1 = (batched_res['country_with_imgs']['Japan'][0])[0][0]

print(np.sum(test))
np.sum(test1)

np.mean(test1)

In [ ]:
old_sum, new_sum = 0,0
for country in old_res['country_with_imgs']:
    for local in range(2):
        num_img_old = len(old_res['country_with_imgs'][country][local])
        num_img_new = len(batched_res['country_with_imgs'][country][local])
        num_images_match = num_img_old == num_img_new
        if not num_images_match:
            print("NUM IMAGES DONT MATCH")
        
        for i in range(num_img_new):
            # check if filepath is same
            filepath_comp = old_res['country_with_imgs'][country][local][i][1] == batched_res['country_with_imgs'][country][local][i][1] 
            if not filepath_comp:
                print("FILEPATHS DONT MATCH")
                
            # check if feature array is same
            old_feat = old_res['country_with_imgs'][country][local][i][0]
            new_feat = batched_res['country_with_imgs'][country][local][i][0]
            
            allclose = np.allclose(old_feat, new_feat, atol = 0.01)
            if not allclose:
                print("Over threshold!")
#             print("element wise comparison thresh  = 0.01: {0}".format(allclose))
            
#             print(np.sum(old_feat), np.sum(new_feat))
            old_sum += np.sum(old_feat)
            new_sum += np.sum(new_feat)
            
#     print("old: sum of all img feature vecs for {0}: {1}".format(country, old_sum))
#     print("batched: sum of all img feature vecs for {0}: {1}".format(country, new_sum))
    if np.abs(old_sum - new_sum) > 0.5:
        print("difference in sum of feature vecs greater than 0.5")



In [ ]:
for i, (data, target) in enumerate(tqdm(dataloader)):
    if i > 3:
        break
    print(torch.sum(data))

In [8]:
import numpy as np
import pickle
import fasttext
from collections import Counter
import re
from countryinfo import CountryInfo
import torchvision.transforms as transforms
from torchvision import models
import torch.utils.data # changed to prevent namespace pollution
import torch.nn.functional as F
import torch.nn as nn
import torch
import pycountry
import copy
from tqdm import tqdm

def country_to_iso3(country):
    missing = {'South+Korea': 'KOR',
            'North+Korea': 'PRK',
            'Laos': 'LAO',
            'Caribbean+Netherlands': 'BES',
            'St.+Lucia': 'LCA',
            'East+Timor': 'TLS',
            'Democratic+Republic+of+Congo': 'COD',
            'Swaziland': 'SWZ',
            'Cape+Verde': 'CPV',
            'C%C3%B4te+d%C2%B4Ivoire': 'CIV',
            'Ivory+Coast': 'CIV',
            'Channel+Islands': 'GBR'
            }
    try:
        iso3 = pycountry.countries.search_fuzzy(country.replace('+', ' '))[0].alpha_3
    except LookupError:
        try:
            iso3 = missing[country]
        except KeyError:
            iso3 = None
    return iso3

def count_country(dataloader, args):
    counts = {}

    for i, (data, target) in enumerate(tqdm(dataloader)):
        country = target[2][0]
        if country not in counts.keys():
            counts[country] = 0
        counts[country] += 1

    pickle.dump(counts, open("results/{}/5.pkl".format(args.folder), "wb"))

def count_tags(dataloader, args):
    country_tags = {}
    tag_to_subregion_features = {}
    categories = dataloader.dataset.categories
    iso3_to_subregion = pickle.load(open('util_files/iso3_to_subregion_mappings.pkl', 'rb'))
    unique_subregions = set(list(iso3_to_subregion.values()))

    # Extracts features from model pretrained on ImageNet
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    model = models.alexnet(pretrained=True).to(device)
    new_classifier = nn.Sequential(*list(model.classifier.children())[:-1])
    model.classifier = new_classifier
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

    subregion_features = {}
    for subregion in unique_subregions:
        subregion_features[subregion] = []
    for cat in range(len(categories)):
        tag_to_subregion_features[cat] = copy.deepcopy(subregion_features)
    for i, (data, target) in enumerate(tqdm(dataloader)):
        if data is None:
            continue
        country = target[2][0]
        anns = target[0]
        filepath = target[3]
        this_categories = list(set([categories.index(ann['label']) for ann in anns]))
        subregion = iso3_to_subregion[country_to_iso3(country)]
        if country not in country_tags.keys():
            country_tags[country] = np.zeros(len(categories))
        this_features = None
        for cat in this_categories:
            if len(tag_to_subregion_features[cat][subregion]) < 500:
                data = normalize(data).to(device)
                big_data = F.interpolate(data.unsqueeze(0), size=224, mode='bilinear').to(device)
                this_features = model.forward(big_data)
                break
        for cat in this_categories:
            country_tags[country][cat] += 1
            if this_features is not None and len(tag_to_subregion_features[cat][subregion]) < 500:
                tag_to_subregion_features[cat][subregion].append((this_features.data.cpu().numpy(), filepath))

    info_stats = {}
    info_stats['country_tags'] = country_tags
    info_stats['tag_to_subregion_features'] = tag_to_subregion_features
    pickle.dump(info_stats, open("results/{}/6.pkl".format(args.folder), "wb"))

def count_langs_batch(dataloader, batch_size = 64):
    print("batch method")
    mappings = pickle.load(open('country_lang_mappings.pkl', 'rb'))
    iso3_to_lang = mappings['iso3_to_lang']
    # Country to iso3 mappings that are missing
    missing = {'South+Korea': 'KOR',
            'North+Korea': 'PRK',
            'Laos': 'LAO',
            'Caribbean+Netherlands': 'BES',
            'St.+Lucia': 'LCA',
            'East+Timor': 'TLS',
            'Democratic+Republic+of+Congo': 'COD',
            'Swaziland': 'SWZ',
            'Cape+Verde': 'CPV',
            'C%C3%B4te+d%C2%B4Ivoire': 'CIV',
            'Ivory+Coast': 'CIV',
            'Channel+Islands': 'GBR'
            }


    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    model = models.alexnet(pretrained=True).to(device)
    new_classifier = nn.Sequential(*list(model.classifier.children())[:-1])
    model.classifier = new_classifier
    # IMPORANT: turn off all model stochasticity
    model.eval()

    with_country = dataloader.dataset.with_country

    country_with_langs = {}
    country_with_imgs = {} # for each country, first list is tourist second is local
    lang_counts = {}
    
    # arrays to cache data for batching speedup
    batched_big_data =[]
    batched_country_data=[]
    batched_local_lang_data=[]
    batched_target_data=[]
    
    
    num_data = len(dataloader)

    detecter = fasttext.load_model('lid.176.bin')
    lang_dict = {}
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

    print(batch_size)
    with torch.no_grad():
        for i, (data, target) in enumerate(tqdm(dataloader)):
            if data is None:
                continue
            this_tags = [tag['label'] for tag in target[0] if len(tag['label']) >= 3]
            if len(this_tags) > 0:
                srcz = []
                conf = []
                for tag in this_tags:
                    classify = detecter.predict(tag)
                    srcz.append(classify[0][0][9:])
                    conf.append(classify[1][0])

                # Pick out the most common language
                commons = Counter(srcz).most_common()
                the_src = commons[0][0]
                # If the most common language is English, look at the second most common language
                # since people oftentimes use English even when it's not their native language
                if the_src == 'en' and len(commons) > 1:
                    the_src_maybe = commons[1][0]
                    words = [i for i in range(len(srcz)) if srcz[i] == the_src_maybe]
                    # If this second most common language has been classified with more than .5
                    # probability, then choose this language for the image
                    for word in words:
                        if conf[word] > .5: 
                            the_src = the_src_maybe
                if the_src in lang_counts.keys():
                    lang_counts[the_src] += 1
                else:
                    lang_counts[the_src] = 1

                country = target[2][0]
                iso3 = None
                local = None
                try:
                    iso3 = pycountry.countries.search_fuzzy(country.replace('+', ' '))[0].alpha_3
                except LookupError:
                    iso3 = missing[country]
                try:
                    country_info = CountryInfo(country.replace('+', ' ')).info()
                except KeyError:
                    country_info = {}
                country_name = country.split('+')
                if 'name' in country_info.keys():
                    country_name += country_info['name']
                if 'nativeName' in country_info.keys():
                    country_name += country_info['nativeName']

                # When comparing images to distinguish between tourist and local, we further look into the content of the tags,
                # allowing some images to be categorized as 'unknown' if we are not that sure if it's tourist or local

                # Local: in a local language, country's name isn't a tag, and 'travel' isn't a tag
                # Tourist: in a non-local language, or 'travel' is a tag
                try:
                    if the_src in iso3_to_lang[iso3] and len(set(country_name)&set(this_tags)) == 0 and 'travel' not in this_tags:
                        local = 1
                    elif the_src not in iso3_to_lang[iso3] or 'travel' in this_tags:
                        local = 0
                except KeyError:
                     print("This iso3 can't be found in iso3_to_lang: {}".format(iso3))

                if country not in country_with_langs.keys():
                    country_with_langs[country] = []
                    country_with_imgs[country] = [[], []]
                country_with_langs[country].append(the_src)

                if local is not None:
                    if len(country_with_imgs[country][local]) < 500:
                        data = normalize(data).to(device)
                        big_data = F.interpolate(data.unsqueeze(0), size=224, mode='bilinear').to(device)

                        batched_big_data.append(big_data)
                        batched_country_data.append(country)
                        batched_local_lang_data.append(local)
                        batched_target_data.append(target[3])

                        if i % batch_size == 0 or i == num_data-1:
                            all_big_data = torch.cat(batched_big_data, 0)
                            batched_features = model(all_big_data)
                            for i, cur_feature in enumerate(batched_features):
                                cur_country = batched_country_data[i]
                                cur_local = batched_local_lang_data[i]
                                cur_target = batched_target_data[i]
                                country_with_imgs[cur_country][cur_local].append((cur_feature.data.cpu().numpy(), cur_target))

                            batched_big_data =[]
                            batched_country_data=[]
                            batched_local_lang_data=[]
                            batched_target_data=[]


    info = {}
    info['lang_counts'] = lang_counts
    info['country_with_langs'] = country_with_langs
    info['country_with_imgs'] = country_with_imgs
    

    pickle.dump(info, open("results/testing/batched_10.pkl", "wb"))
    return info

In [9]:
def count_langs_old(dataloader):
    print("old way no batch")
    mappings = pickle.load(open('country_lang_mappings.pkl', 'rb'))
    iso3_to_lang = mappings['iso3_to_lang']
    # Country to iso3 mappings that are missing
    missing = {'South+Korea': 'KOR',
            'North+Korea': 'PRK',
            'Laos': 'LAO',
            'Caribbean+Netherlands': 'BES',
            'St.+Lucia': 'LCA',
            'East+Timor': 'TLS',
            'Democratic+Republic+of+Congo': 'COD',
            'Swaziland': 'SWZ',
            'Cape+Verde': 'CPV',
            'C%C3%B4te+d%C2%B4Ivoire': 'CIV',
            'Ivory+Coast': 'CIV',
            'Channel+Islands': 'GBR'
            }


    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    model = models.alexnet(pretrained=True).to(device)
    new_classifier = nn.Sequential(*list(model.classifier.children())[:-1])
    model.classifier = new_classifier
    model.eval()

    with_country = dataloader.dataset.with_country

    country_with_langs = {}
    country_with_imgs = {} # for each country, first list is tourist second is local
    lang_counts = {}

    detecter = fasttext.load_model('lid.176.bin')
    lang_dict = {}
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

    for i, (data, target) in enumerate(tqdm(dataloader)):
        if data is None:
            continue
        this_tags = [tag['label'] for tag in target[0] if len(tag['label']) >= 3]
        if len(this_tags) > 0:
            srcz = []
            conf = []
            for tag in this_tags:
                classify = detecter.predict(tag)
                srcz.append(classify[0][0][9:])
                conf.append(classify[1][0])

            # Pick out the most common language
            commons = Counter(srcz).most_common()
            the_src = commons[0][0]
            # If the most common language is English, look at the second most common language
            # since people oftentimes use English even when it's not their native language
            if the_src == 'en' and len(commons) > 1:
                the_src_maybe = commons[1][0]
                words = [i for i in range(len(srcz)) if srcz[i] == the_src_maybe]
                # If this second most common language has been classified with more than .5
                # probability, then choose this language for the image
                for word in words:
                    if conf[word] > .5: 
                        the_src = the_src_maybe
            if the_src in lang_counts.keys():
                lang_counts[the_src] += 1
            else:
                lang_counts[the_src] = 1

            country = target[2][0]
            iso3 = None
            local = None
            try:
                iso3 = pycountry.countries.search_fuzzy(country.replace('+', ' '))[0].alpha_3
            except LookupError:
                iso3 = missing[country]
            try:
                country_info = CountryInfo(country.replace('+', ' ')).info()
            except KeyError:
                country_info = {}
            country_name = country.split('+')
            if 'name' in country_info.keys():
                country_name += country_info['name']
            if 'nativeName' in country_info.keys():
                country_name += country_info['nativeName']

            # When comparing images to distinguish between tourist and local, we further look into the content of the tags,
            # allowing some images to be categorized as 'unknown' if we are not that sure if it's tourist or local

            # Local: in a local language, country's name isn't a tag, and 'travel' isn't a tag
            # Tourist: in a non-local language, or 'travel' is a tag
            try:
                if the_src in iso3_to_lang[iso3] and len(set(country_name)&set(this_tags)) == 0 and 'travel' not in this_tags:
                    local = 1
                elif the_src not in iso3_to_lang[iso3] or 'travel' in this_tags:
                    local = 0
            except KeyError:
                 print("This iso3 can't be found in iso3_to_lang: {}".format(iso3))

            if country not in country_with_langs.keys():
                country_with_langs[country] = []
                country_with_imgs[country] = [[], []]
            country_with_langs[country].append(the_src)
            if local is not None:
                if len(country_with_imgs[country][local]) < 500:
                    data = normalize(data).to(device)
                    big_data = F.interpolate(data.unsqueeze(0), size=224, mode='bilinear').to(device)
                    this_features = model(big_data)
                    country_with_imgs[country][local].append((this_features.data.cpu().numpy(), target[3]))


    info = {}
    info['lang_counts'] = lang_counts
    info['country_with_langs'] = country_with_langs
    info['country_with_imgs'] = country_with_imgs

    pickle.dump(info, open("results/testing/old_10.pkl", "wb"))
    return info